In [14]:
import re
import os
import csv
import pandas as pd

In [2]:
from bs4 import BeautifulSoup
import urllib.request

In [5]:
decades = ['1950s', '1960s', '1970s', '1980s', '1990s', '2000s', '2010s']
decades_dict = {}

In [4]:
# Takes in one of the decade CSV show lists and returns it as a formatted dataframe
def csvToDataframe(filename):
    df = pd.read_csv(filename, header=None, names=['title', 'start', 'end', 'wiki_url', 'transcript_url'])
    return df

In [7]:
for decade in decades:
    path = "src/" + decade[:-1] + "_shows.csv"
    decades_dict[decade] = csvToDataframe(path)
    print(path + " has been processed")

src/1950_shows.csv has been processed
src/1960_shows.csv has been processed
src/1970_shows.csv has been processed
src/1980_shows.csv has been processed
src/1990_shows.csv has been processed
src/2000_shows.csv has been processed
src/2010_shows.csv has been processed


In [9]:
print(decades_dict)

{'1950s':          title      start       end  \
0  I Love Lucy  Oct. 1951  May 1957   

                                    wiki_url  \
0  https://en.wikipedia.org/wiki/I_Love_Lucy   

                                      transcript_url  
0  https://www.springfieldspringfield.co.uk/episo...  , '1960s':                 title      start        end  \
0     The Flintstones  Sep. 1960  Apr. 1966   
1           Bewitched  Sep. 1964  Mar. 1972   
2   The Addams Family  Sep. 1964  Apr. 1966   
3        The Munsters  Sep. 1964   May 1966   
4           Get Smart  Sep. 1965   May 1970   
5  I Dream of Jeannie  Sep. 1965   May 1970   

                                            wiki_url  \
0      https://en.wikipedia.org/wiki/The_Flintstones   
1            https://en.wikipedia.org/wiki/Bewitched   
2  https://en.wikipedia.org/wiki/The_Addams_Famil...   
3         https://en.wikipedia.org/wiki/The_Munsters   
4            https://en.wikipedia.org/wiki/Get_Smart   
5   https://en.wikipedia.org

In [11]:
shows = []
for decade in decades:
    curr_df = decades_dict[decade]
    for i, row in curr_df.iterrows():
        shows.append(row['title'])

In [13]:
print(len(shows))

43


In [15]:
# -*- coding: utf-8 -*-
import re
alphabets= "([A-Za-z])"
prefixes = "(Mr|St|Mrs|Ms|Dr)[.]"
suffixes = "(Inc|Ltd|Jr|Sr|Co)"
starters = "(Mr|Mrs|Ms|Dr|Prof|Capt|Cpt|Lt|He\s|She\s|It\s|They\s|Their\s|Our\s|We\s|But\s|However\s|That\s|This\s|Wherever)"
acronyms = "([A-Z][.][A-Z][.](?:[A-Z][.])?)"
websites = "[.](com|net|org|io|gov|edu|me)"
digits = "([0-9])"
multiple_dots = r'\.{2,}'

def split_into_sentences(text) -> list:
    """
    Split the text into sentences.

    If the text contains substrings "<prd>" or "<stop>", they would lead 
    to incorrect splitting because they are used as markers for splitting.

    :param text: text to be split into sentences
    :type text: str

    :return: list of sentences
    :rtype: list[str]
    """
    text = " " + text + "  "
    text = text.replace("\n"," ")
    text = re.sub(prefixes,"\\1<prd>",text)
    text = re.sub(websites,"<prd>\\1",text)
    text = re.sub(digits + "[.]" + digits,"\\1<prd>\\2",text)
    text = re.sub(multiple_dots, lambda match: "<prd>" * len(match.group(0)) + "<stop>", text)
    if "Ph.D" in text: text = text.replace("Ph.D.","Ph<prd>D<prd>")
    text = re.sub("\s" + alphabets + "[.] "," \\1<prd> ",text)
    text = re.sub(acronyms+" "+starters,"\\1<stop> \\2",text)
    text = re.sub(alphabets + "[.]" + alphabets + "[.]" + alphabets + "[.]","\\1<prd>\\2<prd>\\3<prd>",text)
    text = re.sub(alphabets + "[.]" + alphabets + "[.]","\\1<prd>\\2<prd>",text)
    text = re.sub(" "+suffixes+"[.] "+starters," \\1<stop> \\2",text)
    text = re.sub(" "+suffixes+"[.]"," \\1<prd>",text)
    text = re.sub(" " + alphabets + "[.]"," \\1<prd>",text)
    if "”" in text: text = text.replace(".”","”.")
    if "\"" in text: text = text.replace(".\"","\".")
    if "!" in text: text = text.replace("!\"","\"!")
    if "?" in text: text = text.replace("?\"","\"?")
    text = text.replace(".",".<stop>")
    text = text.replace("?","?<stop>")
    text = text.replace("!","!<stop>")
    text = text.replace("<prd>",".")
    sentences = text.split("<stop>")
    sentences = [s.strip() for s in sentences]
    if sentences and not sentences[-1]: sentences = sentences[:-1]
    return sentences

In [16]:
def clean(txt):
    stripped_txt = re.sub(r' +', ' ', txt)
    transcript = re.sub(r"[\(\[].*?[\)\]]", "", stripped_txt)
    remove_upper = re.sub(r'\b[A-Z]+\b\:', '', transcript)
    sep = os.linesep.join([s for s in remove_upper.splitlines() if s])
    result = split_into_lines(sep)
    return result.strip()

In [17]:
seid = "s01e14"
print("s = " + seid[1:3])
print("e = " + seid[4:])

s = 01
e = 14


In [47]:
# For a dataframe with the columns 'title' and 'transcript_url'
# Processes all shows in the dataframe, getting their episode urls, SEID's, and respective season and episode numbers
# Returns this data as a new dataframe dictionary, with the key being the show's title
# (this df will be used for populating each season's corpus, which will be placed inside the respective decade folder)
# (so this should be organized by decade)
def getEpisodes(df):
    result = pd.DataFrame(columns=['title', 'season', 'episode', 'url', 'seid'])
    for idx, row in df.iterrows():
        curr_show, curr_url = row['title'], row['transcript_url']
        page = urllib.request.urlopen(curr_url)
        soup = BeautifulSoup(page, "lxml")
        seid_list, s_list, e_list = [], [], []
        url_list = []

        for link in soup.find_all('a'):
            route = link.get('href')
            if route.startswith("view_episode_scripts"):
                seid = route[-6:]
                seid_list.append(seid)
                s_list.append(seid[1:3])
                e_list.append(seid[4:])
                url_list.append(route)
        show = pd.DataFrame(columns=['title', 'season', 'episode', 'url', 'seid'])
        show['seid'] = seid_list
        show['url'] = url_list
        show['episode'] = e_list
        show['season'] = s_list
        show['title'] = curr_show
        result = result.append(show, ignore_index=True)
    return result
        
    # return pd.DataFrame({'season': s_list, 'episode': e_list, 'url': url_list, 'seid': seid_list})

In [48]:
episodes_1950 = getEpisodes(decades_dict['1950s'])

In [49]:
episodes_1950.sample(5)

,title,season,episode,url,seid
130,I Love Lucy,05,02,view_episode_scripts.php?tv-show=i-love-lucy-1...,s05e02
16,I Love Lucy,01,16,view_episode_scripts.php?tv-show=i-love-lucy-1...,s01e16
95,I Love Lucy,03,28,view_episode_scripts.php?tv-show=i-love-lucy-1...,s03e28
176,I Love Lucy,06,22,view_episode_scripts.php?tv-show=i-love-lucy-1...,s06e22
10,I Love Lucy,01,10,view_episode_scripts.php?tv-show=i-love-lucy-1...,s01e10


In [50]:
episodes_1960 = getEpisodes(decades_dict['1960s'])
episodes_1970 = getEpisodes(decades_dict['1970s'])
episodes_1980 = getEpisodes(decades_dict['1980s'])
episodes_1990 = getEpisodes(decades_dict['1990s'])
episodes_2000 = getEpisodes(decades_dict['2000s'])
episodes_2010 = getEpisodes(decades_dict['2010s'])

In [51]:
episodes_1950.name = 'episodes_1950'
episodes_1960.name = 'episodes_1960'
episodes_1970.name = 'episodes_1970'
episodes_1980.name = 'episodes_1980'
episodes_1990.name = 'episodes_1990'
episodes_2000.name = 'episodes_2000'
episodes_2010.name = 'episodes_2010'

In [52]:
to_csv = [episodes_1950, episodes_1960, episodes_1970, episodes_1980, episodes_1990, episodes_2000, episodes_2010]

for d in to_csv:
    filename = d.name + '.csv'
    d.to_csv(filename)

In [ ]:
# def get transcripts():

#base = "https://www.springfieldspringfield.co.uk/"
        # curr_show, curr_url = row['title'], base + row['transcript_url']
        # response = urllib.request.urlopen(curr_url)
        # soup = BeautifulSoup(response, 'html.parser')
        # raw_txt = soup.find("div", {"class":"scrolling-script-container"}).get_text()
        # transcript = clean(raw_txt)